## Optimizing tracker parameters by Evolutionary algorithm 

In [85]:
# single gene: [t_high, t_low, t_new, t_match] - t_buff is not very important
# average time for one video process ~ 1 min 10 sec


def evolutionary_algorithm(number_of_generations=20, population_size=10):
    current_generation = 0
    list_of_fitnesses = []
    min_values_list = []
    operand_type = []
    crossover_type = []
    mutation_type = []
    crossover_value = []
    mutation_value = []
    counter = 0

    parameters_population = initialize_population(population_size)

    for parameters_set in parameters_population:
        tracker_results = process_video(parameters_set)
        list_of_fitnesses.append(operative_function(tracker_results))
    print(list_of_fitnesses)

    current_min_value = min(list_of_fitnesses)
    best_individual_idx = list_of_fitnesses.index(current_min_value)
    best_individual = parameters_population[best_individual_idx]
    min_values_list.append(current_min_value)

    while current_generation <= number_of_generations:
        new_population = []
        fitness_table_for_current_population = []

        # selecting
        selected_population, list_of_fitnesses = select_population(parameters_population=parameters_population, list_of_fitnesses=list_of_fitnesses)
        while True:
            if len(new_population) == len(parameters_population):
                break
            break
        current_generation += 1

evolutionary_algorithm()

[189.13735327391768, 289.81084569053434]


In [192]:
import numpy as np
import sys
import os
import ruamel.yaml
import random
from ruamel.yaml import YAML
from copy import copy


def select_population(parameters_population, list_of_fitnesses, selection_size=60):
    selected_population = []
    selected_fitnesses = []
    list_of_weights = [1/i for i in list_of_fitnesses]
    selection_size = int(len(parameters_population)*selection_size/100)

    copy_population = copy(parameters_population)

    for i in range(selection_size):
        selected_individual = random.choices(copy_population, weights=list_of_weights, k=1)

        selected_population.append(selected_individual[0])
        idx_selected = copy_population.index(selected_individual[0])
        selected_fitnesses.append(list_of_fitnesses[idx_selected])

        copy_population.remove(selected_individual[0])
        list_of_fitnesses.pop(idx_selected)
        list_of_weights.pop(idx_selected)

    return selected_population, list_of_fitnesses


def initialize_population(population_num):
    parameters_names = ["track_high_thresh", "track_low_thresh", "new_track_thresh", "match_thresh"]
    parameters_population = []
    for i in range(population_num):
        parameters_set = {}
        for name in parameters_names:
            parameters_set[name] = round(random.random(), 4)
        parameters_population.append(parameters_set)
    return parameters_population


def operative_function(params:dict):
    ID_COUNT = 100
    TOTAL_LIFE = 2200
    AVG_LIFE = 22.4
    MAX_ID_COUNT = 100
    result = (abs(params["id_max"]-MAX_ID_COUNT)/MAX_ID_COUNT) * (1 + (abs(params["id_count"]-ID_COUNT)/ID_COUNT)) * \
             (1 + (abs(params["total_life"]-TOTAL_LIFE)/TOTAL_LIFE)) *\
             (1 + (abs(params["avg_life"]-AVG_LIFE)/AVG_LIFE)) * (1/params["avg_iou"])
    return result
    

def process_video(parameters_set:dict):
    yaml = YAML()
    with open('../trackers/tracker.yaml', 'r') as fp:
        data = yaml.load(fp)
    data["track_high_thresh"] = parameters_set["track_high_thresh"]
    data['track_low_thresh'] = parameters_set["track_low_thresh"]
    data['new_track_thresh'] = parameters_set["new_track_thresh"]
    data['match_thresh'] = parameters_set["match_thresh"]

    with open('../trackers/tracker.yaml', 'w') as file:
        yaml.dump(data, file)

        track_history, iou_values = get_tracker_results()

        if len(track_history) >= 1:
            id_max = max(track_history.keys())
            id_count = len(track_history.keys())
            tot_life = 0
            for el in track_history.values():
                tot_life += len(el)
            avg_life = tot_life/id_count
            avg_iou = np.nanmean(iou_values)
        else:
            id_max = np.inf
            id_count = np.finfo(float).eps
            tot_life = np.finfo(float).eps
            avg_life = np.finfo(float).eps
            avg_iou = np.finfo(float).eps

    return {'id_max':id_max, 'id_count':id_count, 'total_life':tot_life, 'avg_life':avg_life, 'avg_iou':avg_iou}
    

# pop = initialize_population(4)
# print(pop)

# sample_indicators = {'id_max':200, 'id_count':110, 'total_life':1700, 'avg_life':27, 'avg_iou':0.73}
# sample_indicators_1 = {'id_max':100, 'id_count':100, 'total_life':2200, 'avg_life':20, 'avg_iou':0.79}

# print(operative_function(sample_indicators))
# print(operative_function(sample_indicators_1))

# pop = [[0.1, 0.1, 0.1, 0.1], [0.2, 0.2, 0.2, 0.2], [0.3, 0.3, 0.3, 0.4], [0.5, 0.5, 0.5, 0.5], [0.7, 0.7, 0.7, 0.7]]
# fit = [143, 193, 243, 50, 100]

# print(select_population(parameters_population=pop, list_of_fitnesses=fit))

[0.006993006993006993, 0.0051813471502590676, 0.00411522633744856, 0.02, 0.01]
[[0.5, 0.5, 0.5, 0.5]]
[0.006993006993006993, 0.0051813471502590676, 0.00411522633744856, 0.01]
[[0.2, 0.2, 0.2, 0.2]]
[0.006993006993006993, 0.00411522633744856, 0.01]
[[0.3, 0.3, 0.3, 0.4]]
([[0.5, 0.5, 0.5, 0.5], [0.2, 0.2, 0.2, 0.2], [0.3, 0.3, 0.3, 0.4]], [143, 100])


In [61]:
import cv2
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from natsort import natsorted
from ouster import client
from ouster import pcap
from ouster.client import Scans, XYZLut, SensorInfo, destagger
from ouster.client.data import LidarScan
from ultralytics import YOLO
from contextlib import closing
from collections import defaultdict

def get_xyxy_from_txt(label):
    gt_boxes = list(map(float, label.split(' ')[1:]))
    x_center = gt_boxes[0]*1024
    y_center = gt_boxes[1]*128
    box_width = gt_boxes[2]*1024
    box_height = gt_boxes[3]*128
    
    x1 = int(x_center - (box_width / 2))
    y1 = int(y_center - (box_height / 2))
    x2 = int(x_center + (box_width / 2))
    y2 = int(y_center + (box_height / 2))

    return [x1, y1, x2, y2]


def calculate_iou(box1, box2):
    x1_box1, y1_box1, x2_box1, y2_box1 = box1
    x1_box2, y1_box2, x2_box2, y2_box2 = box2

    x_left = max(x1_box1, x1_box2)
    y_top = max(y1_box1, y1_box2)
    x_right = min(x2_box1, x2_box2)
    y_bottom = min(y2_box1, y2_box2)
    
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    
    area_box1 = (x2_box1 - x1_box1) * (y2_box1 - y1_box1)
    area_box2 = (x2_box2 - x1_box2) * (y2_box2 - y1_box2)
    
    union_area = area_box1 + area_box2 - intersection_area
    
    iou = intersection_area / union_area if union_area > 0 else 0
    return iou


def associate_boxes(boxes1, boxes2):
    associations = []
    for box1 in boxes1:
        max_iou = 0
        associated_box = None
        for box2 in boxes2:
            iou = calculate_iou(box1, box2)
            if iou > max_iou:
                max_iou = iou
                associated_box = box2
        associations.append((box1, associated_box, max_iou))
    return associations


def get_tracker_results():

    output_dict = {3:["GO AHEAD", (0, 255, 0)], 2:["BE CAREFUL", (0, 155, 100)], 1:["SLOW DOWN", (0, 100, 155)], 
                    0:["BREAK", (0, 0, 255)]}

    # Load the YOLOv8 model
    model = YOLO('../weights/best_3000_n_200.pt')

    # Paths to pcap and json files
    metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
    pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

    # Making PacketSource from data
    with open(metadata_path, 'r') as f:
        metadata = client.SensorInfo(f.read())

    fps = int(str(metadata.mode)[-2:])
    width = int(str(metadata.mode)[:4])
    height = int(str(metadata.prod_line)[5:])

    pcap_file = pcap.Pcap(pcap_path, metadata)

    xyz_lut = client.XYZLut(metadata) #call cartesian lookup table

    # Store the track history
    track_history = defaultdict(lambda: [])     # dictionary: {key=id, value=[xyz_val_0, ... ,xyz_val_99]}
    box_history = defaultdict(lambda: [])
    # iou_values = []
    iou_values = np.full(1, np.nan)

    directory_path = 'groundtruthing_labels/'

    with closing(Scans(pcap_file)) as scans:

        save_path = "C:/Users/szyme/Ouster/Dangerous-situations-with-pedastrians/results_mp4"
        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        files = os.listdir(directory_path)
        files_sorted = natsorted(files)

        i = 0
        for scan, filename in zip(scans, files_sorted):
            i += 1

            f = os.path.join(directory_path, filename)
            if os.path.isfile(f):
                labels_file = f
            with open(labels_file, 'r') as file:
                labels = file.readlines()

            sig_field = scan.field(client.ChanField.SIGNAL)
            sig_destaggered = destagger(metadata, sig_field)
            scaling_factor = 0.004
            scaled_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered)
            signal_image = scaled_arr.astype(np.uint8)
            combined_img = np.dstack((signal_image, signal_image, signal_image))

            xyz_destaggered = client.destagger(metadata, xyz_lut(scan))

            results = model.track(source=combined_img, persist=True, imgsz=1024, tracker='../trackers/tracker.yaml', verbose=False)

            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

            if (results[0].boxes.id == None):
                ids = ''
            else:
                ids = results[0].boxes.id.cpu().numpy().astype(int)

            priority = 3
            distance = 0

            gt_boxes = []
            for label in labels:
                gt_boxes.append(get_xyxy_from_txt(label))

            associations = associate_boxes(gt_boxes, boxes)
            for box1, box2, iou in associations:
                if isinstance(box2, np.ndarray):
                    iou_values = np.append(iou_values, iou)
                    # iou_values.append(iou)
                else:
                    iou_values = np.append(iou_values, np.nan)

            for box, id in zip(boxes, ids):

                center_x, center_y = int((box[0] + box[2])/2), int((box[1] + box[3])/2) #

                xyz_val = xyz_destaggered[(center_y, center_x)] #get the (x,y,z) coordinates with the lookup table 

                track = track_history[id] #save the (x,y,z) coordinates for distance calculation
                track.append(xyz_val)

                cv2.rectangle(combined_img, (box[0], box[1]), (box[2], box[3]), output_dict[priority][1], 2)
                cv2.rectangle(combined_img, (box[0], box[1]+2), (box[0]+20+len(str(id))*11, box[1]-12), (255, 255, 255), -1)
                cv2.putText(combined_img, f"Id {id}", (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            cv2.putText(combined_img, f"{output_dict[priority][0]}", (470, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, output_dict[priority][1], 2)

            if True:
                cv2.imshow("YOLOv8 Tracking", combined_img)
                cv2.waitKey(1)  # 1 millisecond

            vid_writer.write(combined_img)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            if i >= 1000:
                break

        vid_writer.release()
        cv2.destroyAllWindows()
    return track_history, np.array(iou_values)